# Apply the union, joins and etc in Pyspark

In [2]:
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### Create a Spark Sessions

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.getOrCreate()

### Create the dataframe with the same file

In [4]:
df = spark.read.csv('C:/Users/Gregory Toledo/learn_spark/wc2018-players.csv', header = True, inferSchema=True)

### Applying the most important changes equal than first notebook for use in the functions correctly

In [5]:
df = df.withColumnRenamed('Pos.', 'Position')
df = df.withColumnRenamed('#', 'Number')
df = df.withColumn('Year', substring('Birth Date', -4, 4))
df = df.withColumn('Year', column('Year').cast(IntegerType()))

day = udf(lambda date: date.split('.')[0])
month = udf(lambda date: date.split('.')[1])
year = udf(lambda date: date.split('.')[2])

df = df.withColumn('Birth Date', concat_ws('-',year('Birth Date'),month('Birth Date'),day('Birth Date')))
df = df.withColumn('Birth Date', column('Birth Date').cast(DateType()))

In [6]:
df.show(5)

+---------+------+--------+------------------+----------+----------+--------------------+------+------+----+
|     Team|Number|Position| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|Year|
+---------+------+--------+------------------+----------+----------+--------------------+------+------+----+
|Argentina|     3|      DF|TAGLIAFICO Nicolas|1992-08-31|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|1992|
|Argentina|    22|      MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|    65|1996|
|Argentina|    15|      MF|    LANZINI Manuel|1993-02-15|   LANZINI|West Ham United F...|   167|    66|1993|
|Argentina|    18|      DF|    SALVIO Eduardo|1990-07-13|    SALVIO|    SL Benfica (POR)|   167|    69|1990|
|Argentina|    10|      FW|      MESSI Lionel|1987-06-24|     MESSI|  FC Barcelona (ESP)|   170|    72|1987|
+---------+------+--------+------------------+----------+----------+--------------------+------+------+----+
only showing top 5 

## Distinct()

In [7]:
df.select('Team').distinct().show(50)

+--------------+
|          Team|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
|  Saudi Arabia|
|   Switzerland|
|        Brazil|
|         Japan|
|       England|
|        Poland|
|      Portugal|
|     Australia|
|    Costa Rica|
|         Egypt|
|        Serbia|
|      Colombia|
+--------------+



## Collect()

In [8]:
list1 = df.select('Team').distinct().collect()

In [9]:
list1[0]
#Row

Row(Team='Russia')

In [10]:
list1[0][0]
#item

'Russia'

In [11]:
#List comprehension that take the items on the rows of list1
countries = [country[0] for country in list1]

In [12]:
print(countries)

['Russia', 'Senegal', 'Sweden', 'IR Iran', 'Germany', 'France', 'Argentina', 'Belgium', 'Peru', 'Croatia', 'Nigeria', 'Korea Republic', 'Spain', 'Denmark', 'Morocco', 'Panama', 'Iceland', 'Uruguay', 'Mexico', 'Tunisia', 'Saudi Arabia', 'Switzerland', 'Brazil', 'Japan', 'England', 'Poland', 'Portugal', 'Australia', 'Costa Rica', 'Egypt', 'Serbia', 'Colombia']


## When() / Otherwise()

In [13]:
# Similar then if/else

df.withColumn('New_Column', when(col('Team') == 'Brazil', 'Brazilian').otherwise('Verify')).show(5)

+---------+------+--------+------------------+----------+----------+--------------------+------+------+----+----------+
|     Team|Number|Position| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|Year|New_Column|
+---------+------+--------+------------------+----------+----------+--------------------+------+------+----+----------+
|Argentina|     3|      DF|TAGLIAFICO Nicolas|1992-08-31|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|1992|    Verify|
|Argentina|    22|      MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|    65|1996|    Verify|
|Argentina|    15|      MF|    LANZINI Manuel|1993-02-15|   LANZINI|West Ham United F...|   167|    66|1993|    Verify|
|Argentina|    18|      DF|    SALVIO Eduardo|1990-07-13|    SALVIO|    SL Benfica (POR)|   167|    69|1990|    Verify|
|Argentina|    10|      FW|      MESSI Lionel|1987-06-24|     MESSI|  FC Barcelona (ESP)|   170|    72|1987|    Verify|
+---------+------+--------+-------------

In [14]:
europe = ['Sweden', 'Germany', 'France', 'Belgium', 'Croatia', 'Spain', 'Denmark', 'Iceland', 'Switzerland', 'England', 'Poland','Portugal','Serbia']
asia = ['Russia', 'IR Iran', 'Korea Republic', 'Saudi Arabia', 'Japan']
oceania = ['Australia']
africa = ['Nigeria' , 'Morocco', 'Senegal', 'Tunisia', 'Egypt']
north_america = ['Panama', 'Mexico', 'Costa Rica']
south_america = ['Argentina', 'Brazil', 'Uruguay', 'Colombia', 'Peru']

In [15]:
df = df.withColumn('Continent', when(col('Team').isin(asia), 'Asian')\
              .when(col('Team').isin(oceania), 'oceanic')\
              .when(col('Team').isin(africa), 'African')\
              .when(col('Team').isin(europe), 'european')\
              .when(col('Team').isin(south_america), 'South American')\
              .when(col('Team').isin(north_america), 'North American')\
              .otherwise('Verify'))

## Union()

##### This function need to have 2 df's with the same amount of colmuns

In [16]:
df_south_america = df.where('Continent = "South American"')
df_north_america = df.where('Continent = "North American"')

In [17]:
df_america = df_south_america.union(df_north_america)

In [18]:
df_america.select('Team').distinct().show()

+----------+
|      Team|
+----------+
| Argentina|
|      Peru|
|   Uruguay|
|    Brazil|
|  Colombia|
|    Panama|
|    Mexico|
|Costa Rica|
+----------+



## Joins

In [36]:
arg = df.where('Team == "Argentina"')
br = df.where('Team == "Brazil"')
arg = arg.drop('Club', 'Year', 'Continent', 'Weight','Birth Date')
br = br.drop('Club', 'Year', 'Continent', 'Weight','Birth Date')

#### Simple Join

In [38]:
newdf = arg.join(br, arg.Number == br.Number)

In [39]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|  Team|Number|Position|FIFA Popular Name| Shirt Name|Height|
+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|Argentina|     4|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|   169|Brazil|     4|      DF|    PEDRO GEROMEL|    GEROMEL|   190|
|Argentina|    23|      MF|    PAVON Cristian|     PAVÓN|   169|Brazil|    23|      GK|          EDERSON|    EDERSON|   188|
|Argentina|    16|      MF|    LANZINI Manuel|   LANZINI|   167|Brazil|    16|      GK|           CASSIO|     CASSIO|   195|
|Argentina|    19|      DF|    SALVIO Eduardo|    SALVIO|   167|Brazil|    19|      MF|          WILLIAN|    WILLIAN|   175|
|Argentina|    11|      FW|      MESSI Lionel|     MESSI|   170|Brazil|    11|      MF|PHILIPPE COUTINHO|P. COUTINHO|   172|


#### Inner Join

In [ ]:
#Alter the numbers of Argentina to use the joins correctly
arg = arg.withColumn('Number', col('Number')+1)

In [41]:
newdf = arg.join(br, arg.Number == br.Number, 'inner')

In [42]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|  Team|Number|Position|FIFA Popular Name| Shirt Name|Height|
+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|Argentina|     4|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|   169|Brazil|     4|      DF|    PEDRO GEROMEL|    GEROMEL|   190|
|Argentina|    23|      MF|    PAVON Cristian|     PAVÓN|   169|Brazil|    23|      GK|          EDERSON|    EDERSON|   188|
|Argentina|    16|      MF|    LANZINI Manuel|   LANZINI|   167|Brazil|    16|      GK|           CASSIO|     CASSIO|   195|
|Argentina|    19|      DF|    SALVIO Eduardo|    SALVIO|   167|Brazil|    19|      MF|          WILLIAN|    WILLIAN|   175|
|Argentina|    11|      FW|      MESSI Lionel|     MESSI|   170|Brazil|    11|      MF|PHILIPPE COUTINHO|P. COUTINHO|   172|


#### Left Join

In [43]:
newdf = arg.join(br, arg.Number == br.Number, 'left')

In [44]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|  Team|Number|Position|FIFA Popular Name| Shirt Name|Height|
+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|Argentina|     4|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|   169|Brazil|     4|      DF|    PEDRO GEROMEL|    GEROMEL|   190|
|Argentina|    23|      MF|    PAVON Cristian|     PAVÓN|   169|Brazil|    23|      GK|          EDERSON|    EDERSON|   188|
|Argentina|    16|      MF|    LANZINI Manuel|   LANZINI|   167|Brazil|    16|      GK|           CASSIO|     CASSIO|   195|
|Argentina|    19|      DF|    SALVIO Eduardo|    SALVIO|   167|Brazil|    19|      MF|          WILLIAN|    WILLIAN|   175|
|Argentina|    11|      FW|      MESSI Lionel|     MESSI|   170|Brazil|    11|      MF|PHILIPPE COUTINHO|P. COUTINHO|   172|


#### Right Join

In [45]:
newdf = arg.join(br, arg.Number == br.Number, 'right')

In [46]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|  Team|Number|Position|FIFA Popular Name| Shirt Name|Height|
+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|Argentina|    18|      DF|  OTAMENDI Nicolas|  OTAMENDI|   181|Brazil|    18|      MF|             FRED|       FRED|   169|
|Argentina|    21|      MF|  LO CELSO Giovani|  LO CELSO|   177|Brazil|    21|      FW|           TAISON|     TAISON|   172|
|Argentina|    17|      DF|       ROJO Marcos|      ROJO|   189|Brazil|    17|      MF|      FERNANDINHO|FERNANDINHO|   179|
|Argentina|    22|      FW|      DYBALA Paulo|    DYBALA|   177|Brazil|    22|      DF|           FAGNER|     FAGNER|   168|
|Argentina|    10|      FW|   HIGUAIN Gonzalo|   HIGUAÍN|   184|Brazil|    10|      FW|           NEYMAR|  NEYMAR JR|   175|


#### Full Join

In [47]:
newdf = arg.join(br, arg.Number == br.Number, 'full')

In [48]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|  Team|Number|Position|FIFA Popular Name| Shirt Name|Height|
+---------+------+--------+------------------+----------+------+------+------+--------+-----------------+-----------+------+
|     null|  null|    null|              null|      null|  null|Brazil|     1|      GK|          ALISSON|  A. BECKER|   193|
|Argentina|     2|      GK|     GUZMAN Nahuel|    GUZMÁN|   192|Brazil|     2|      DF|     THIAGO SILVA|   T. SILVA|   183|
|Argentina|     3|      DF|   MERCADO Gabriel|   MERCADO|   181|Brazil|     3|      DF|          MIRANDA|    MIRANDA|   186|
|Argentina|     4|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|   169|Brazil|     4|      DF|    PEDRO GEROMEL|    GEROMEL|   190|
|Argentina|     5|      DF|  ANSALDI Cristian|   ANSALDI|   181|Brazil|     5|      MF|         CASEMIRO|   CASEMIRO|   185|


#### Semi Join

In [49]:
newdf = arg.join(br, arg.Number == br.Number, 'semi')

In [50]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|
+---------+------+--------+------------------+----------+------+
|Argentina|     4|      DF|TAGLIAFICO Nicolas|TAGLIAFICO|   169|
|Argentina|    23|      MF|    PAVON Cristian|     PAVÓN|   169|
|Argentina|    16|      MF|    LANZINI Manuel|   LANZINI|   167|
|Argentina|    19|      DF|    SALVIO Eduardo|    SALVIO|   167|
|Argentina|    11|      FW|      MESSI Lionel|     MESSI|   170|
|Argentina|     5|      DF|  ANSALDI Cristian|   ANSALDI|   181|
|Argentina|     6|      MF|      BIGLIA Lucas|    BIGLIA|   175|
|Argentina|     8|      MF|       BANEGA Ever|    BANEGA|   175|
|Argentina|    15|      DF| MASCHERANO Javier|MASCHERANO|   174|
|Argentina|    22|      FW|      DYBALA Paulo|    DYBALA|   177|
|Argentina|    20|      FW|     AGUERO Sergio|    AGÜERO|   172|
|Argentina|    10|      FW|   HIGUAIN Gonzalo|   HIGUAÍN|   184|
|Argentina|    12|      M

#### Anti Join

In [51]:
newdf = arg.join(br, arg.Number == br.Number, 'anti')

In [52]:
newdf.show(25)

+---------+------+--------+------------------+----------+------+
|     Team|Number|Position| FIFA Popular Name|Shirt Name|Height|
+---------+------+--------+------------------+----------+------+
|Argentina|    24|      GK|CABALLERO Wilfredo| CABALLERO|   186|
+---------+------+--------+------------------+----------+------+

